In [1]:
import numpy as np
import pandas as pd
from datetime import datetime as dt
import itertools

%matplotlib inline 

In [2]:
#Reading the data

loc = "G:\Projects\Football-Analysis\MyFootball\Dataset"
rawData1 = pd.read_csv(loc + '\\2017-18.csv')
rawData2 = pd.read_csv(loc + '\\2018-19.csv')

In [3]:
#Parse data as time

def parse_date(date):
    if(date==''):
        return None
    else:
        return dt.strptime(date, '%d/%m/%Y').date()
    
rawData1.Date=rawData1.Date.apply(parse_date)
rawData2.Date=rawData2.Date.apply(parse_date)

#Getting all the required stats data form the table and creating another table

columnsReq=['Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR']
playingStats1=rawData1[columnsReq]
playingStats2=rawData2[columnsReq]

In [4]:
playingStats1.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,2017-08-11,Arsenal,Leicester,4,3,H
1,2017-08-12,Brighton,Man City,0,2,A
2,2017-08-12,Chelsea,Burnley,2,3,A
3,2017-08-12,Crystal Palace,Huddersfield,0,3,A
4,2017-08-12,Everton,Stoke,1,0,H


## GOALS SCORED AND CONCEDED AT THE END OF A MATCHWEEK, ARRANGED AS PER TEAMS AND MATCH WEEK

In [5]:
#FTHG = Full Time Home Team Goals 
#FTAG = Full Time Away Team Goals 

#Gets agg goals scored arranged as per teams and matchweek
def getGoalsScored(playing_stat):
    #dictionrary with team name as keys
    teams={}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
        
    #values corresponding to keys is a list of match location
    for i in range(len(playing_stat)):
        HTGS = playing_stat.iloc[i]['FTHG'] #HOME TEAM GOALS SCORED
        ATGS = playing_stat.iloc[i]['FTAG']
        teams[playing_stat.iloc[i].HomeTeam].append(HTGS)
        teams[playing_stat.iloc[i].AwayTeam].append(ATGS)
    
    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    GoalsScored = pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T
    GoalsScored[0] = 0
    
    # Aggregate to get uptil that point
    for i in range(2,39):
        GoalsScored[i] = GoalsScored[i] + GoalsScored[i-1]
    return GoalsScored

# Gets the goals conceded agg arranged by teams and matchweek
def getGoalsConceded(playing_stat):
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    
    # the value corresponding to keys is a list containing the match location.
    for i in range(len(playing_stat)):
        ATGC = playing_stat.iloc[i]['FTHG'] #AWAY TEAM GOALS CONCEDED
        HTGC = playing_stat.iloc[i]['FTAG']
        teams[playing_stat.iloc[i].HomeTeam].append(HTGC)
        teams[playing_stat.iloc[i].AwayTeam].append(ATGC)
    
    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    GoalsConceded = pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T
    GoalsConceded[0] = 0
    # Aggregate to get uptil that point
    for i in range(2,39):
        GoalsConceded[i] = GoalsConceded[i] + GoalsConceded[i-1]
    return GoalsConceded

def get_gss(playing_stat):
    GC = getGoalsConceded(playing_stat)
    GS = getGoalsScored(playing_stat)
   
    j = 0
    HTGS = []
    ATGS = []
    HTGC = []
    ATGC = []

    for i in range(380):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        HTGS.append(GS.loc[ht][j])
        ATGS.append(GS.loc[at][j])
        HTGC.append(GC.loc[ht][j])
        ATGC.append(GC.loc[at][j])
        
        if ((i + 1)% 10) == 0:
            j = j + 1
        
    playing_stat['HTGS'] = HTGS
    playing_stat['ATGS'] = ATGS
    playing_stat['HTGC'] = HTGC
    playing_stat['ATGC'] = ATGC
    
    return playing_stat


# Apply to each dataset
playingStats1 = get_gss(playingStats1)
playingStats2 = get_gss(playingStats2)

C:\Users\munis_000\Anaconda3\lib\site-packages\ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\munis_000\Anaconda3\lib\site-packages\ipykernel_launcher.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\munis_000\Anaconda3\lib\site-packages\ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing

In [6]:
playingStats1

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,ATGC
0,2017-08-11,Arsenal,Leicester,4,3,H,0,0,0,0
1,2017-08-12,Brighton,Man City,0,2,A,0,0,0,0
2,2017-08-12,Chelsea,Burnley,2,3,A,0,0,0,0
3,2017-08-12,Crystal Palace,Huddersfield,0,3,A,0,0,0,0
4,2017-08-12,Everton,Stoke,1,0,H,0,0,0,0
5,2017-08-12,Southampton,Swansea,0,0,D,0,0,0,0
6,2017-08-12,Watford,Liverpool,3,3,D,0,0,0,0
7,2017-08-12,West Brom,Bournemouth,1,0,H,0,0,0,0
8,2017-08-13,Man United,West Ham,4,0,H,0,0,0,0
9,2017-08-13,Newcastle,Tottenham,0,2,A,0,0,0,0


In [7]:
def get_points(result):
    if result == 'W':
        return 3
    elif result == 'D':
        return 1
    else:
        return 0
    

def get_cuml_points(matchres):
    matchres_points = matchres.applymap(get_points)
    for i in range(2,39):
        matchres_points[i] = matchres_points[i] + matchres_points[i-1]
        
    matchres_points.insert(column =0, loc = 0, value = [0*i for i in range(20)])
    return matchres_points


def get_matchres(playing_stat):
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []

    # the value corresponding to keys is a list containing the match result
    for i in range(len(playing_stat)):
        if playing_stat.iloc[i].FTR == 'H':
            teams[playing_stat.iloc[i].HomeTeam].append('W')
            teams[playing_stat.iloc[i].AwayTeam].append('L')
        elif playing_stat.iloc[i].FTR == 'A':
            teams[playing_stat.iloc[i].AwayTeam].append('W')
            teams[playing_stat.iloc[i].HomeTeam].append('L')
        else:
            teams[playing_stat.iloc[i].AwayTeam].append('D')
            teams[playing_stat.iloc[i].HomeTeam].append('D')
            
    return pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T

def getAggPoints(playing_stat):
    matchres = get_matchres(playing_stat)
    cum_pts = get_cuml_points(matchres)
    HTP = [] #Away Team Points
    ATP = [] #Home Team Points
    j = 0
    for i in range(380):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        HTP.append(cum_pts.loc[ht][j])
        ATP.append(cum_pts.loc[at][j])

        if ((i + 1)% 10) == 0:
            j = j + 1
            
    playing_stat['HTP'] = HTP
    playing_stat['ATP'] = ATP
    return playing_stat
    
# Apply to each dataset
playingStats1 = getAggPoints(playingStats1)
playingStats2 = getAggPoints(playingStats2)

C:\Users\munis_000\Anaconda3\lib\site-packages\ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\munis_000\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
playingStats1

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,ATGC,HTP,ATP
0,2017-08-11,Arsenal,Leicester,4,3,H,0,0,0,0,0,0
1,2017-08-12,Brighton,Man City,0,2,A,0,0,0,0,0,0
2,2017-08-12,Chelsea,Burnley,2,3,A,0,0,0,0,0,0
3,2017-08-12,Crystal Palace,Huddersfield,0,3,A,0,0,0,0,0,0
4,2017-08-12,Everton,Stoke,1,0,H,0,0,0,0,0,0
5,2017-08-12,Southampton,Swansea,0,0,D,0,0,0,0,0,0
6,2017-08-12,Watford,Liverpool,3,3,D,0,0,0,0,0,0
7,2017-08-12,West Brom,Bournemouth,1,0,H,0,0,0,0,0,0
8,2017-08-13,Man United,West Ham,4,0,H,0,0,0,0,0,0
9,2017-08-13,Newcastle,Tottenham,0,2,A,0,0,0,0,0,0


## GET TEAM FORM:

In [9]:
def get_form(playing_stat,num):
    form = get_matchres(playing_stat)
    form_final = form.copy()
    for i in range(num,39):
        form_final[i] = ''
        j = 0
        while j < num:
            form_final[i] += form[i-j]
            j += 1           
    return form_final

def add_form(playing_stat,num):
    form = get_form(playing_stat,num)
    h = ['M' for i in range(num * 10)]  # since form is not available for n MW (n*10)
    a = ['M' for i in range(num * 10)]
    
    j = num
    for i in range((num*10),380):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        
        past = form.loc[ht][j]               # get past n results
        h.append(past[num-1])                    # 0 index is most recent
        
        past = form.loc[at][j]               # get past n results.
        a.append(past[num-1])                   # 0 index is most recent
        
        if ((i + 1)% 10) == 0:
            j = j + 1

    playing_stat['HM' + str(num)] = h                 
    playing_stat['AM' + str(num)] = a

    
    return playing_stat


def add_form_df(playing_statistics):
    playing_statistics = add_form(playing_statistics,1)
    playing_statistics = add_form(playing_statistics,2)
    playing_statistics = add_form(playing_statistics,3)
    playing_statistics = add_form(playing_statistics,4)
    playing_statistics = add_form(playing_statistics,5)
    return playing_statistics    
    
# Make changes to df
playingStats1= add_form_df(playingStats1)
playingStats2= add_form_df(playingStats2)


C:\Users\munis_000\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\munis_000\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [10]:
# Rearranging columns
cols = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTGS', 'ATGS', 'HTGC', 'ATGC', 'HTP', 'ATP', 'HM1', 'HM2', 'HM3',
        'HM4', 'HM5', 'AM1', 'AM2', 'AM3', 'AM4', 'AM5' ]
playingStats1= playingStats1[cols]
playingStats2= playingStats2[cols]

playingStats1


,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,ATGC,...,HM1,HM2,HM3,HM4,HM5,AM1,AM2,AM3,AM4,AM5
0,2017-08-11,Arsenal,Leicester,4,3,H,0,0,0,0,...,M,M,M,M,M,M,M,M,M,M
1,2017-08-12,Brighton,Man City,0,2,A,0,0,0,0,...,M,M,M,M,M,M,M,M,M,M
2,2017-08-12,Chelsea,Burnley,2,3,A,0,0,0,0,...,M,M,M,M,M,M,M,M,M,M
3,2017-08-12,Crystal Palace,Huddersfield,0,3,A,0,0,0,0,...,M,M,M,M,M,M,M,M,M,M
4,2017-08-12,Everton,Stoke,1,0,H,0,0,0,0,...,M,M,M,M,M,M,M,M,M,M
5,2017-08-12,Southampton,Swansea,0,0,D,0,0,0,0,...,M,M,M,M,M,M,M,M,M,M
6,2017-08-12,Watford,Liverpool,3,3,D,0,0,0,0,...,M,M,M,M,M,M,M,M,M,M
7,2017-08-12,West Brom,Bournemouth,1,0,H,0,0,0,0,...,M,M,M,M,M,M,M,M,M,M
8,2017-08-13,Man United,West Ham,4,0,H,0,0,0,0,...,M,M,M,M,M,M,M,M,M,M
9,2017-08-13,Newcastle,Tottenham,0,2,A,0,0,0,0,...,M,M,M,M,M,M,M,M,M,M


## Get Last Year's Position as also an independent variable:

In [11]:
Standings = pd.read_csv(loc + "\\EPLStandings.csv")
Standings.set_index(['Team'], inplace=True)
Standings = Standings.fillna(18)

def get_last(playing_stat, Standings, year):
    HomeTeamLP = []
    AwayTeamLP = []
    for i in range(380):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        HomeTeamLP.append(Standings.loc[ht][year])
        AwayTeamLP.append(Standings.loc[at][year])
    playing_stat['HomeTeamLP'] = HomeTeamLP
    playing_stat['AwayTeamLP'] = AwayTeamLP
    return playing_stat

playingStats1 = get_last(playingStats1, Standings, 0)
playingStats2 = get_last(playingStats2, Standings, 0)



C:\Users\munis_000\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\munis_000\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [12]:
playingStats1

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,ATGC,...,HM3,HM4,HM5,AM1,AM2,AM3,AM4,AM5,HomeTeamLP,AwayTeamLP
0,2017-08-11,Arsenal,Leicester,4,3,H,0,0,0,0,...,M,M,M,M,M,M,M,M,6.0,4.0
1,2017-08-12,Brighton,Man City,0,2,A,0,0,0,0,...,M,M,M,M,M,M,M,M,15.0,2.0
2,2017-08-12,Chelsea,Burnley,2,3,A,0,0,0,0,...,M,M,M,M,M,M,M,M,18.0,18.0
3,2017-08-12,Crystal Palace,Huddersfield,0,3,A,0,0,0,0,...,M,M,M,M,M,M,M,M,18.0,16.0
4,2017-08-12,Everton,Stoke,1,0,H,0,0,0,0,...,M,M,M,M,M,M,M,M,18.0,18.0
5,2017-08-12,Southampton,Swansea,0,0,D,0,0,0,0,...,M,M,M,M,M,M,M,M,19.0,3.0
6,2017-08-12,Watford,Liverpool,3,3,D,0,0,0,0,...,M,M,M,M,M,M,M,M,20.0,1.0
7,2017-08-12,West Brom,Bournemouth,1,0,H,0,0,0,0,...,M,M,M,M,M,M,M,M,13.0,18.0
8,2017-08-13,Man United,West Ham,4,0,H,0,0,0,0,...,M,M,M,M,M,M,M,M,18.0,18.0
9,2017-08-13,Newcastle,Tottenham,0,2,A,0,0,0,0,...,M,M,M,M,M,M,M,M,18.0,14.0


## GET MATCHWEEK

In [16]:
def get_mw(playing_stat):
    j = 1
    MatchWeek = []
    for i in range(380):
        MatchWeek.append(j)
        if ((i + 1)% 10) == 0:
            j = j + 1
    playing_stat['MW'] = MatchWeek
    return playing_stat

playingStats1 = get_mw(playingStats1)
playingStats2 = get_mw(playingStats2)

C:\Users\munis_000\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [17]:
playingStats1

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,ATGC,...,HM4,HM5,AM1,AM2,AM3,AM4,AM5,HomeTeamLP,AwayTeamLP,MW
0,2017-08-11,Arsenal,Leicester,4,3,H,0,0,0,0,...,M,M,M,M,M,M,M,6.0,4.0,1
1,2017-08-12,Brighton,Man City,0,2,A,0,0,0,0,...,M,M,M,M,M,M,M,15.0,2.0,1
2,2017-08-12,Chelsea,Burnley,2,3,A,0,0,0,0,...,M,M,M,M,M,M,M,18.0,18.0,1
3,2017-08-12,Crystal Palace,Huddersfield,0,3,A,0,0,0,0,...,M,M,M,M,M,M,M,18.0,16.0,1
4,2017-08-12,Everton,Stoke,1,0,H,0,0,0,0,...,M,M,M,M,M,M,M,18.0,18.0,1
5,2017-08-12,Southampton,Swansea,0,0,D,0,0,0,0,...,M,M,M,M,M,M,M,19.0,3.0,1
6,2017-08-12,Watford,Liverpool,3,3,D,0,0,0,0,...,M,M,M,M,M,M,M,20.0,1.0,1
7,2017-08-12,West Brom,Bournemouth,1,0,H,0,0,0,0,...,M,M,M,M,M,M,M,13.0,18.0,1
8,2017-08-13,Man United,West Ham,4,0,H,0,0,0,0,...,M,M,M,M,M,M,M,18.0,18.0,1
9,2017-08-13,Newcastle,Tottenham,0,2,A,0,0,0,0,...,M,M,M,M,M,M,M,18.0,14.0,1


## FINAL DATAFRAME

In [18]:
playing_stat= pd.concat([playingStats1,
                          playingStats2], ignore_index=True)


# Gets the form points.
def get_form_points(string):
    sum = 0
    for letter in string:
        sum += get_points(letter)
    return sum

playing_stat['HTFormPtsStr'] = playing_stat['HM1'] + playing_stat['HM2'] + playing_stat['HM3'] + playing_stat['HM4'] + playing_stat['HM5']
playing_stat['ATFormPtsStr'] = playing_stat['AM1'] + playing_stat['AM2'] + playing_stat['AM3'] + playing_stat['AM4'] + playing_stat['AM5']

playing_stat['HTFormPts'] = playing_stat['HTFormPtsStr'].apply(get_form_points)
playing_stat['ATFormPts'] = playing_stat['ATFormPtsStr'].apply(get_form_points)

# Identify Win/Loss Streaks if any.
def get_3game_ws(string):
    if string[-3:] == 'WWW':
        return 1
    else:
        return 0
    
def get_5game_ws(string):
    if string == 'WWWWW':
        return 1
    else:
        return 0
    
def get_3game_ls(string):
    if string[-3:] == 'LLL':
        return 1
    else:
        return 0
    
def get_5game_ls(string):
    if string == 'LLLLL':
        return 1
    else:
        return 0
    
playing_stat['HTWinStreak3'] = playing_stat['HTFormPtsStr'].apply(get_3game_ws)
playing_stat['HTWinStreak5'] = playing_stat['HTFormPtsStr'].apply(get_5game_ws)
playing_stat['HTLossStreak3'] = playing_stat['HTFormPtsStr'].apply(get_3game_ls)
playing_stat['HTLossStreak5'] = playing_stat['HTFormPtsStr'].apply(get_5game_ls)

playing_stat['ATWinStreak3'] = playing_stat['ATFormPtsStr'].apply(get_3game_ws)
playing_stat['ATWinStreak5'] = playing_stat['ATFormPtsStr'].apply(get_5game_ws)
playing_stat['ATLossStreak3'] = playing_stat['ATFormPtsStr'].apply(get_3game_ls)
playing_stat['ATLossStreak5'] = playing_stat['ATFormPtsStr'].apply(get_5game_ls)

playing_stat.keys()

Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTGS', 'ATGS',
       'HTGC', 'ATGC', 'HTP', 'ATP', 'HM1', 'HM2', 'HM3', 'HM4', 'HM5', 'AM1',
       'AM2', 'AM3', 'AM4', 'AM5', 'HomeTeamLP', 'AwayTeamLP', 'MW',
       'HTFormPtsStr', 'ATFormPtsStr', 'HTFormPts', 'ATFormPts',
       'HTWinStreak3', 'HTWinStreak5', 'HTLossStreak3', 'HTLossStreak5',
       'ATWinStreak3', 'ATWinStreak5', 'ATLossStreak3', 'ATLossStreak5'],
      dtype='object')

In [19]:
# Get Goal Difference
playing_stat['HTGD'] = playing_stat['HTGS'] - playing_stat['HTGC']
playing_stat['ATGD'] = playing_stat['ATGS'] - playing_stat['ATGC']

# Diff in points
playing_stat['DiffPts'] = playing_stat['HTP'] - playing_stat['ATP']
playing_stat['DiffFormPts'] = playing_stat['HTFormPts'] - playing_stat['ATFormPts']

# Diff in last year positions
playing_stat['DiffLP'] = playing_stat['HomeTeamLP'] - playing_stat['AwayTeamLP']

In [20]:
# Scale DiffPts , DiffFormPts, HTGD, ATGD by Matchweek.
cols = ['HTGD','ATGD','DiffPts','DiffFormPts','HTP','ATP']
playing_stat.MW = playing_stat.MW.astype(float)

for col in cols:
    playing_stat[col] = playing_stat[col] / playing_stat.MW

In [21]:
def only_hw(string):
    if string == 'H':
        return 'H'
    else:
        return 'NH'
    
playing_stat['FTR'] = playing_stat.FTR.apply(only_hw)

# Testing set (2015-16 season)
playing_stat_test = playing_stat[650:]

In [28]:
columns=playing_stat.columns.tolist()
len(columns)

42

In [29]:
playing_stat.tail()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,ATGC,...,HTLossStreak5,ATWinStreak3,ATWinStreak5,ATLossStreak3,ATLossStreak5,HTGD,ATGD,DiffPts,DiffFormPts,DiffLP
755,2019-05-12,Liverpool,Wolves,2,0,H,87,47,22,44,...,0,0,0,0,0,1.710526,0.078947,0.973684,0.131579,-17.0
756,2019-05-12,Man United,Cardiff,0,2,NH,65,32,52,69,...,0,0,0,0,0,0.342105,-0.973684,0.921053,0.052632,0.0
757,2019-05-12,Southampton,Huddersfield,1,1,NH,44,21,64,75,...,0,0,0,1,0,-0.526316,-1.421053,0.605263,0.105263,3.0
758,2019-05-12,Tottenham,Everton,2,2,NH,65,52,37,44,...,0,0,0,0,0,0.736842,0.210526,0.447368,-0.105263,-4.0
759,2019-05-12,Watford,West Ham,1,4,NH,51,48,55,54,...,0,0,0,0,0,-0.105263,-0.157895,0.026316,-0.078947,2.0


In [30]:
playing_stat.to_csv(loc + "\\final_dataset.csv")
playing_stat_test.to_csv(loc+"\\test.csv")